In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

# Assuming you have already loaded the BACI data
BACI_PATH = "path/to/your/BACI_HS92_V202201.csv"
df = pd.read_csv(BACI_PATH)

# Rename columns for clarity (as in the previous example)
df = df.rename(columns={
    't': 'year',
    'i': 'exporter',
    'j': 'importer',
    'k': 'product_code',
    'v': 'trade_value_thousands_usd',
    'q': 'quantity_tons'
})

df['trade_value_millions_usd'] = df['trade_value_thousands_usd'] / 1000

In [2]:
import pandas as pd
import glob
import os
from tqdm import tqdm
import numpy as np

# Define paths
csv_directory = '/Users/ian/Downloads/TradeThesis_Helfrich_Final/data/raw/cepii/cepii_baci_hs02_v202401b/'
output_file = 'sampled_baci_data.csv'

# Get list of CSV files, excluding country and product code files
csv_files = [f for f in glob.glob(
    f'{csv_directory}*.csv') if 'country_codes' not in f and 'product_codes' not in f]

# Load country and product code data
country_codes = pd.read_csv(f'{csv_directory}country_codes_V202401b.csv')
product_codes = pd.read_csv(f'{csv_directory}product_codes_HS02_V202401b.csv')

# Function to process a chunk of data


def process_chunk(chunk, country_codes, product_codes):
    # Merge with country codes (for both i and j)
    chunk = pd.merge(chunk, country_codes, left_on='i',
                     right_on='country_code', how='left')
    chunk = pd.merge(chunk, country_codes, left_on='j',
                     right_on='country_code', how='left', suffixes=('_i', '_j'))

    # Merge with product codes
    chunk = pd.merge(chunk, product_codes, left_on='k',
                     right_on='product_code', how='left')

    return chunk

# Function to sample 1% of data


def sample_chunk(chunk, sample_frac=0.01):
    return chunk.sample(frac=sample_frac)


# Initialize empty DataFrame for the sample
sampled_data = pd.DataFrame()

# Process files
total_size = sum(os.path.getsize(f) for f in csv_files)
with tqdm(total=total_size, unit='B', unit_scale=True, desc="Processing files") as pbar:
    for file in csv_files:
        # Extract year from filename
        year = int(file.split('_Y')[-1].split('_')[0])

        # Read and process the file in chunks
        for chunk in pd.read_csv(file, chunksize=100000):
            chunk['year'] = year
            processed_chunk = process_chunk(
                chunk, country_codes, product_codes)
            sampled_chunk = sample_chunk(processed_chunk)
            sampled_data = pd.concat(
                [sampled_data, sampled_chunk], ignore_index=True)

            pbar.update(chunk.memory_usage(deep=True).sum())

# Save the sampled data
sampled_data.to_csv(output_file, index=False)
print(f"Sampled data (1%) saved to '{output_file}'")

# Display summary of the sampled data
print("\nSampled Data Summary:")
print(sampled_data.info())
print("\nSample of the data:")
print(sampled_data.head())

Processing files:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

Processing files:   0%|          | 0.00/9.85G [00:00<?, ?B/s]


KeyError: 'product_code'

In [3]:
import pandas as pd
import glob
import os
from tqdm import tqdm
import numpy as np


def analyze_csv(file_path):
    """Analyze a CSV file and return its structure."""
    df = pd.read_csv(
        file_path, nrows=1000)  # Read first 1000 rows for analysis
    return {
        'columns': list(df.columns),
        'dtypes': df.dtypes.to_dict(),
        'sample': df.head(5).to_dict()
    }


def find_merge_keys(main_structure, auxiliary_structure):
    """Find potential merge keys between main and auxiliary dataframes."""
    main_cols = set(main_structure['columns'])
    aux_cols = set(auxiliary_structure['columns'])

    # Find common columns
    common_cols = main_cols.intersection(aux_cols)

    # Check for columns that might be renamed (e.g., 'i' in main, 'country_code' in auxiliary)
    potential_renames = []
    for main_col in main_cols:
        for aux_col in aux_cols:
            if main_structure['dtypes'][main_col] == auxiliary_structure['dtypes'][aux_col]:
                if main_structure['sample'][main_col].values() == auxiliary_structure['sample'][aux_col].values():
                    potential_renames.append((main_col, aux_col))

    return list(common_cols), potential_renames


def process_chunk(chunk, auxiliary_dfs, merge_info):
    """Process a chunk of the main dataframe, merging with auxiliary data."""
    for aux_name, merge_keys in merge_info.items():
        left_on, right_on = merge_keys
        chunk = pd.merge(
            chunk, auxiliary_dfs[aux_name], left_on=left_on, right_on=right_on, how='left')
    return chunk


# Define paths
csv_directory = '/Users/ian/Downloads/TradeThesis_Helfrich_Final/data/raw/cepii/cepii_baci_hs02_v202401b/'
output_file = 'sampled_baci_data.csv'

# Get list of CSV files
csv_files = glob.glob(f'{csv_directory}*.csv')

# Analyze all CSV files
file_structures = {file: analyze_csv(file) for file in csv_files}

# Identify main data files and auxiliary files
main_files = [f for f in csv_files if 'BACI_HS02_Y' in f]
auxiliary_files = [f for f in csv_files if f not in main_files]

# Analyze main data structure (using the first main file)
main_structure = file_structures[main_files[0]]

# Find merge keys for auxiliary files
merge_info = {}
auxiliary_dfs = {}
for aux_file in auxiliary_files:
    aux_structure = file_structures[aux_file]
    common_cols, potential_renames = find_merge_keys(
        main_structure, aux_structure)

    if common_cols:
        merge_info[aux_file] = (common_cols[0], common_cols[0])
    elif potential_renames:
        merge_info[aux_file] = potential_renames[0]

    auxiliary_dfs[aux_file] = pd.read_csv(aux_file)

# Initialize empty DataFrame for the sample
sampled_data = pd.DataFrame()

# Process files
total_size = sum(os.path.getsize(f) for f in main_files)
with tqdm(total=total_size, unit='B', unit_scale=True, desc="Processing files") as pbar:
    for file in main_files:
        # Extract year from filename
        year = int(file.split('_Y')[-1].split('_')[0])

        # Read and process the file in chunks
        for chunk in pd.read_csv(file, chunksize=100000):
            chunk['year'] = year
            processed_chunk = process_chunk(chunk, auxiliary_dfs, merge_info)
            sampled_chunk = processed_chunk.sample(frac=0.01)  # 1% sample
            sampled_data = pd.concat(
                [sampled_data, sampled_chunk], ignore_index=True)

            pbar.update(chunk.memory_usage(deep=True).sum())

# Save the sampled data
sampled_data.to_csv(output_file, index=False)
print(f"Sampled data (1%) saved to '{output_file}'")

# Display summary of the sampled data
print("\nSampled Data Summary:")
print(sampled_data.info())
print("\nSample of the data:")
print(sampled_data.head())

Processing files: 22.7GB [08:50, 42.7MB/s]                            


Sampled data (1%) saved to 'sampled_baci_data.csv'

Sampled Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060489 entries, 0 to 2060488
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   t       int64  
 1   i       int64  
 2   j       int64  
 3   k       int64  
 4   v       float64
 5   q       object 
 6   year    int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 110.0+ MB
None

Sample of the data:
      t   i    j       k       v              q  year
0  2021   8  380  281511  11.488          6.935  2021
1  2021  32  784  848590  12.684          0.058  2021
2  2021  32  214  870870   6.900             NA  2021
3  2021  32   56  871690  11.921          1.750  2021
4  2021   8  616  611120   0.182          0.012  2021


In [1]:
import pandas as pd

# Path to your parquet file
file_path = '/Users/ian/Downloads/TradeThesis_Helfrich_Final/data/raw/cepii/cepii_gravity_loadingCodeExperiment/test11.parquet'

# Read the parquet file
df = pd.read_parquet(file_path)

# Inspect the DataFrame
print("First 5 rows:")
print(df.head())

print("\nData types:")
print(df.dtypes)

print("\nShape of the DataFrame:")
print(df.shape)

print("\nDescriptive statistics:")
print(df.describe())

First 5 rows:
      t  i   j       k       v       q  country_code exporter_name  \
0  2021  4  20  420229   0.598   0.001             4   Afghanistan   
1  2021  4  20  621050   0.200   0.001             4   Afghanistan   
2  2021  4  20  640319   0.167   0.001             4   Afghanistan   
3  2021  4  20  870899   0.461   0.001             4   Afghanistan   
4  2021  4  24  382490  53.015  13.235             4   Afghanistan   

  exporter_iso2 exporter_iso3  country_code_importer importer_name  \
0            AF           AFG                     20       Andorra   
1            AF           AFG                     20       Andorra   
2            AF           AFG                     20       Andorra   
3            AF           AFG                     20       Andorra   
4            AF           AFG                     24        Angola   

  importer_iso2 importer_iso3    code  \
0            AD           AND  420229   
1            AD           AND  621050   
2            AD      